<a href="https://colab.research.google.com/github/weavermarquez/jeopardybot/blob/main/SimpleT5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from huggingface_hub import notebook_login
# notebook_login()

In [ ]:
!pip install -q transformers datasets simplet5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 94.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 49.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 97.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.7/527.7 kB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 69.5 MB/s eta 0:00

# Prepare Data



## (2nd run) Unpickle Processed Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive/') #, force_remount=True)
import os

!ls /content/drive/My\ Drive/Colab\ Notebooks/preprocessed_t5_jeopardy.pkl

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
simplet5-epoch-0-train-loss-3.0885-val-loss-2.7971
simplet5-epoch-2-train-loss-2.6905-val-loss-2.8803
'/content/drive/My Drive/Colab Notebooks/preprocessed_t5_jeopardy.pkl'


In [ ]:
import pickle

drive_dir = '/content/drive/My Drive/Colab Notebooks'
filename = '/preprocessed_t5_jeopardy.pkl'

input_data = None
# Pickle
# with open(drive_dir + filename, 'wb+') as f:
#     pickle.dump(input_data, f)

# Unpickle
with open(drive_dir + filename, 'rb') as f:
    unpickled = pickle.load(f)
    f.close()

train_df, validation_df, test_df = unpickled
train_df.shape, validation_df.shape, test_df.shape

## (1st run) Data Cleaning and Processing

In [ ]:
# Step 1: Convert Jeopardy dataset into text-to-text format, then split into train/validation/test

import pandas as pd
from datasets import load_dataset, load_metric

jeopardy = load_dataset("jeopardy")
jeopardy

### Removing all multimedia samples, Convert to T5 task format

In [ ]:
#  - function(example: Dict) -> bool
def only_text(example):
  # return not (example['question'].contains('audio clue') or example['question'].contains('video clue') or example['question'].contains('a href'))
  # return not ('audio clue' in example['question'] or 'video clue' in example['question'] or 'a href' in example['question'])
  string = example['question']
  substring_list = ['audio_clue', 'video_clue', 'a href']
  return not (any(substring in string for substring in substring_list))

def convert_to_text_to_text(example):
  input_text = f"jeopardy: {example['answer']} in {example['category']} for {example['value']}"
  output_text = example['question']
  return {
    "source_text": input_text,
    "target_text": output_text
  }

jeopardy_cleaned = jeopardy.filter(only_text) #, input_columns = 'question')
# map(convert_to_text_to_text, remove_columns=jeopardy['train'].column_names)
jeopardy_cleaned

# remove_columns=['category','air_date','question','value','answer','round','show_number']
jeopardy_text_to_text = jeopardy_cleaned.map(
    convert_to_text_to_text, remove_columns=jeopardy_cleaned['train'].column_names)
jeopardy_text_to_text

DatasetDict({
    train: Dataset({
        features: ['category', 'air_date', 'question', 'value', 'answer', 'round', 'show_number'],
        num_rows: 206407
    })
})

### Split to DataFrame across train/test/validation


In [ ]:
def convert_datasectdict_to_dataframe(dataset, seed = 42):
  data_test = dataset['train'].train_test_split(test_size=0.1, seed=seed)
  train_validation = data_test['train'].train_test_split(test_size=0.1, seed=seed)

  # optionally: mutate ataset
  # dataset['train'] = train_validation['train']
  # dataset['validation'] = train_validation['test']
  # dataset['test'] = train_test['test']

  train_df = pd.DataFrame(train_validation['train'])
  validation_df = pd.DataFrame(train_validation['test'])
  test_df = pd.DataFrame(data_test['test'])

  return train_df, validation_df, test_df

train_df, validation_df, test_df = convert_datasectdict_to_dataframe(jeopardy_text_to_text)
train_df.shape, validation_df.shape, test_df.shape

((167189, 2), (18577, 2), (20641, 2))

# Train from Pretrained

In [ ]:
# Move back to /content so I dont eradicate my google drive capacity
# os.chdir("/content")
!pwd

!ls

# current saved models
!ls /outputs
print('')

# cloud-storage models
!ls /content/drive/My\ Drive/Colab\ Notebooks/outputs

/content
drive  sample_data

simplet5-epoch-0-train-loss-3.0885-val-loss-2.7971
simplet5-epoch-2-train-loss-2.6905-val-loss-2.8803


In [ ]:
# Copy cloud-storage to current session
# !mkdir /content/outputs
!cp -r -t /content /content/drive/My\ Drive/Colab\ Notebooks/outputs/

# !rm -rf /content/outputs

In [ ]:
from simplet5 import SimpleT5

train_amount = 500
test_amount = 100

model = SimpleT5()

# Clean Model
model.from_pretrained(model_type="t5", model_name="t5-base")

# Continue from Previous Epochs
# finetuned = 'outputs/' + 'simplet5-epoch-0-train-loss-1.8197-val-loss-3.537'
# model.from_pretrained(model_type="t5", model_name=finetuned)

model.train(train_df=train_df[:train_amount],
            eval_df=test_df[:test_amount],
            source_max_token_len=128,
            target_max_token_len=50,
            batch_size=8, max_epochs=10, use_gpu=True)

ModuleNotFoundError: ignored

# Inference

In [ ]:
from simplet5 import SimpleT5
model = SimpleT5()

# let's load the trained model for inferencing:
# finetuned = 'outputs/' + 'simplet5-epoch-2-train-loss-2.6905-val-loss-2.8803'
finetuned = 'outputs/' + 'simplet5-epoch-0-train-loss-3.0885-val-loss-2.7971'
model.load_model("t5",finetuned,use_gpu=True)

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

sample_df =  test_df[100:].sample(10, random_state = 420)
predict = [model.predict(x)[0] for x in sample_df.source_text]
sample_df['generated_text'] = predict

sample_df


# sample_series =  test_df[100:].sample(10, random_state = 42).source_text

,source_text,target_text,generated_text
7140,jeopardy: Stillwater in MAIN STREET U.S.A. for 3200,"'Your feelings may run deep on historic Main Street in this city, home of Oklahoma State University'",'The main street in the city of this is named for a man who lived there from 1850 to 1870'
20583,jeopardy: Demond Wilson in PRESIDENTIAL LAST NAMES for 800,'On TV he played Sanford's son','He was the last president to be named in the U.S. Senate'
6587,jeopardy: Verb in ENGLISH USAGE for 100,"'The predicate is the part of the sentence that contains this & its modifiers, objects & complements'","'It's the verb that precedes ""to"" & ""to""'"
14120,jeopardy: Biafra in THE THIRD WORLD for 400,"'On May 30, 1967, this state declared independence from Nigeria, but surrendered 2 1/2 years later'","'The third largest country in Africa, it's home to the U.N.'"
3372,jeopardy: Velazquez in EUROPEAN ART & ARTISTS for 2000,'Manhattan's Frick collection is home to a 1644 portrait of King Philip IV by this great Spanish artist',"'In the 1890s this Spanish painter was known as ""The Man of the Night""'"
16216,jeopardy: Julius & Ethel Rosenberg in BOOKS & AUTHORS for 500,"'E.L. Doctorow's novel ""The Book of Daniel"" was based on the espionage trial of this couple'","'In ""The Last Days"", this pair tells the story of their first marriage'"
14061,jeopardy: Hungary in THE AGE OF NATIONALISM for 2000,'Lajos Kossuth was representing the county of Pest when he led this country's nationalist revolt in 1848',"'In the 1850s this country was called ""The Great""; in the 19th century it was called ""The Great""'"
20514,"jeopardy: The Great Profile in ""GREAT"" MOVIES for 400",'John Barrymore spoofed himself in this 1940 movie whose title reflected his nickname',"'In this 1993 film, the title character is named for a woman who has been married to a man since his death'"
10285,jeopardy: Marie Louise in THERE'S SOMETHING ABOUT MARIE ANTOINETTE for 800,'Her great-niece who married Napoleon',"'In the 1890s, this Frenchwoman became the first woman to win an Oscar for her role in ""The Greatest Show""'"
16533,jeopardy: CNN in PHRASES THAT SELL for 600,"'This network says it's ""The most trusted name in news""'","'This cable news network has been called ""The World's Largest Television Network""'"


In [ ]:
points = [100, 200, 300, 400, 500, 600, 800, 1000, 2000]
categories = ['GARDENING IN THE SPRING', 'PORTRAITS AND PAINTINGS', 'GIFTS FOR YOUR MOTHER', 'HISTORICAL DOGS', 'I CANT BELIEVE IT\'S NOT REAL']
answers = ['worms', 'soil', 'topsoil', 'shovels', 'peppers', 'tomato', 'birds','compost','trellis']


[[model.predict(f'jeopardy: {answer} in {category} for {point}') for answer, point in zip(answers, points)] for category in categories]

# ------
# my_sample = "jeopardy: Queen Elizabeth in for 1000"
# model.predict(my_sample)

[[["'In the springtime, you can plant these in your garden to prevent them from spreading'"],
  ["'In the spring, you can plant plants in this type of soil'"],
  ["'In the spring, this type of soil can be used to cover plants or as a mulch'"],
  ["'In the spring, gardeners use these tools to shovel their way through the yard'"],
  ['\'The name of this vegetable comes from the Latin word for "spray"\''],
  ["'In the spring, you can plant this vegetable in a pot or on a bed of lettuce; it's also called a tomato'"],
  ["'In the springtime, you can plant these in your garden to help keep them from getting eaten'"],
  ["'In the spring, you can use this type of material to make your lawn look like a garden'"],
  ["'As the name suggests, it's a decorative ornamental plant that can be used to cover your lawn'"]],
 [["'In the early 1900s this insect was painted on a wall of water, but it didn't make its way to the surface'"],
  ["'As the name suggests, it's the surface of a lake or river'"],
  

In [ ]:
# predicted = model.predict(sample_source)

# print(f'prompt: {sample_source}')
# print(f'expected: {expected}')
# print(f'generated: {predicted}')

prompt: jeopardy: Osama bin Laden in WANTED BY THE FBI for 600
expected: 'His FBI listing says he's wanted for 1998 bombings of U.S. embassies & is "a suspect in other terrorist attacks"'
generated: ["'He was wanted by the FBI for his role in the 9/11 attacks'"]


# Check Google Drive Usage

In [ ]:
# !mkdir drive/MyDrive/Colab\ Notebooks/lightning_logs
!cp -r -t drive/MyDrive/Colab\ Notebooks/lightning_logs lightning_logs/version_1
# !du -sh drive/MyDrive/Colab\ Notebooks/outputs/*

In [ ]:
!du -sh outputs/*

854M	outputs/simplet5-epoch-0-train-loss-3.0885-val-loss-2.7971
854M	outputs/simplet5-epoch-0-train-loss-3.2133-val-loss-2.9522
854M	outputs/simplet5-epoch-1-train-loss-2.873-val-loss-2.9061
854M	outputs/simplet5-epoch-2-train-loss-2.6905-val-loss-2.8803


In [ ]:
!du -sh lightning_logs

2.5G	lightning_logs
